In [ ]:
import yfinance as yf


#LOAD ALL DATA
import sqlite3
import pandas as pd
import datetime
import os


dbFile ='.\\NSE_1min_DATA.sqlite.db'

def getSymbolsData(symbol):
    symbol = symbol.upper()
    if not os.path.exists(dbFile):
        print("File does not exist")
        return None
    conn = sqlite3.connect(dbFile)
    #conn = sqlite3.connect('.\\NSE_MASTER_DATA.sqlite.db')
    df = pd.read_sql_query("SELECT  * FROM  '"+symbol+"' ", conn)
    conn.close()
    return df


###
### START FROM HERE 
data = getSymbolsData("FACT-EQ")

 

import pandas as pd
import numpy as np
from nltk import ngrams
from collections import Counter 

# Load your OHLCV data

df2 = data[['Date','time','Open',   'High',    'Low',  'Close',  'Volume' ]].copy()


df = df2.rename(columns={ 'time':'Time'})
# If your columns are named exactly "Date" and "Time"
df['Date_Time'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'].astype(str))



def detect_doji(data, threshold=0.001):
    """
    Detect Doji candlestick patterns.
    :param data: DataFrame with Open, High, Low, Close.
    :param threshold: Percentage threshold for near-equal open and close.
    """
    data['Doji'] = abs(data['Close'] - data['Open']) <= (data['High'] - data['Low']) * threshold
    return data

data = detect_doji(data)
print(data[data['Doji']])  # Print rows where a Doji is detected

def detect_head_and_shoulders(data, lookback=10):
    """
    Detect potential head-and-shoulders patterns.
    :param data: DataFrame with High column.
    :param lookback: Number of periods to consider for peaks.
    """
    from scipy.signal import find_peaks

    # Find peaks (high points)
    peaks, _ = find_peaks(data['High'], distance=lookback)

    # Check for head and shoulders: Three peaks, middle is highest
    patterns = []
    for i in range(1, len(peaks) - 1):
        if data['High'][peaks[i-1]] < data['High'][peaks[i]] > data['High'][peaks[i+1]]:
            patterns.append((peaks[i-1], peaks[i], peaks[i+1]))
    return patterns

patterns = detect_head_and_shoulders(data)
print(f"Detected Head-and-Shoulders Patterns: {patterns}")

def detect_triangle(data, lookback=20):
    """
    Detect potential triangle patterns.
    :param data: DataFrame with High and Low columns.
    :param lookback: Number of periods for support/resistance.
    """
    from scipy.stats import linregress

    # Calculate trend lines
    lows = data['Low'].rolling(window=lookback).min()
    highs = data['High'].rolling(window=lookback).max()

    # Check for converging trend lines
    slopes_low = linregress(range(lookback), lows[-lookback:]).slope
    slopes_high = linregress(range(lookback), highs[-lookback:]).slope

    if slopes_low > 0 and slopes_high < 0 and slopes_low < abs(slopes_high):
        return True  # Triangle pattern detected
    return False

if detect_triangle(data):
    print("Triangle pattern detected!")

In [12]:
df = getSymbolsData("FACT-EQ")
df.columns


Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Date', 'time',
       'Cumulative Volume', 'rvol_Candle', 'rvolTotal'],
      dtype='object')

In [3]:
if detect_triangle(data):
    print("Triangle pattern detected!")

In [10]:
detect_doji(data)
data[data['Doji']==True]

,Open,High,Low,Close,Volume,Date,time,Cumulative Volume,rvol_Candle,rvolTotal,Doji
11,638.30,639.00,636.95,638.30,2108,2025-03-19,09:26:59.000000,62665,1.000000,1.000000,True
29,645.00,645.85,645.00,645.00,1495,2025-03-19,09:44:58.000000,127258,1.000000,1.000000,True
51,640.50,641.05,640.50,640.50,737,2025-03-19,10:06:57.000000,155265,1.000000,1.000000,True
59,638.00,638.00,638.00,638.00,185,2025-03-19,10:14:55.000000,163840,1.000000,1.000000,True
73,640.35,640.35,640.35,640.35,24,2025-03-19,10:28:43.000000,172846,1.000000,1.000000,True
...,...,...,...,...,...,...,...,...,...,...,...
6834,741.95,742.20,741.55,741.95,10078,2025-04-17,12:54:56.000000,5952823,3.543912,3.509879,True
6909,757.70,757.70,756.00,757.70,47767,2025-04-17,14:09:59.000000,9753349,2.974531,2.756515,True
6920,758.30,758.95,757.70,758.30,21681,2025-04-17,14:20:59.000000,10337420,4.264388,4.450626,True
6924,752.00,753.60,751.30,752.00,69103,2025-04-17,14:24:59.000000,10782239,4.843555,4.510547,True
